In [7]:
import json
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD, RDFS, OWL
from neo4j import GraphDatabase

# Define namespaces
EX = Namespace("http://example.org/")

# Neo4j connection
uri = "bolt://localhost:7687"  # Change if necessary
username = "neo4j"  # Replace with your Neo4j username
password = "Neo4jpassword"  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

# Load JSON Data
def load_json_data(financial_file, eod_file):
    with open(financial_file, 'r', encoding='utf-8') as file:
        financial_data = json.load(file)
    with open(eod_file, 'r', encoding='utf-8') as file:
        eod_data = json.load(file)
    return financial_data, eod_data

# Add schema definitions to RDF Graph
def add_schema_definitions(g):
    # Class definitions
    g.add((EX.Company, RDF.type, OWL.Class))
    g.add((EX.Security, RDF.type, OWL.Class))
    g.add((EX.StockValue, RDF.type, OWL.Class))
    g.add((EX.FinancialStatement, RDF.type, OWL.Class))
    g.add((EX.IndustrialClassification, RDF.type, OWL.Class))

    # Property definitions
    g.add((EX.hasPrice, RDF.type, OWL.ObjectProperty))
    g.add((EX.hasPrice, RDFS.domain, EX.Company))
    g.add((EX.hasPrice, RDFS.range, EX.Security))

    g.add((EX.name, RDF.type, OWL.DatatypeProperty))
    g.add((EX.name, RDFS.domain, OWL.Thing))
    g.add((EX.name, RDFS.range, XSD.string))

    g.add((EX.isPriceOf, RDF.type, OWL.ObjectProperty))
    g.add((EX.isPriceOf, OWL.inverseOf, EX.hasPrice))

    # StockValue attributes
    for attribute in ['has_average', 'has_close', 'has_high', 'has_low', 'has_open', 'has_prior', 'has_totalVolume']:
        g.add((getattr(EX, attribute), RDF.type, OWL.DatatypeProperty))
        g.add((getattr(EX, attribute), RDFS.domain, EX.StockValue))
        g.add((getattr(EX, attribute), RDFS.range, XSD.float))

    g.add((EX.hasFinancialStatement, RDF.type, OWL.ObjectProperty))
    g.add((EX.hasFinancialStatement, RDFS.domain, EX.Company))
    g.add((EX.hasFinancialStatement, RDFS.range, EX.FinancialStatement))

    g.add((EX.hasYear, RDF.type, OWL.DatatypeProperty))
    g.add((EX.hasYear, RDFS.domain, EX.FinancialStatement))
    g.add((EX.hasYear, RDFS.range, XSD.integer))

    g.add((EX.hasQuarter, RDF.type, OWL.DatatypeProperty))
    g.add((EX.hasQuarter, RDFS.domain, EX.FinancialStatement))
    g.add((EX.hasQuarter, RDFS.range, XSD.integer))

    # Financial statement properties
    financial_key_list = ["totalAssets", "totalLiabilities", "paidupShareCapital", "shareholderEquity",
                          "totalEquity", "totalRevenueQuarter", "totalRevenueAccum", "totalExpensesQuarter", "totalExpensesAccum",
                          "ebitQuarter", "ebitAccum", "netProfitQuarter", "netProfitAccum", "epsQuarter", "epsAccum",
                          "operatingCashFlow", "investingCashFlow", "financingCashFlow", "roe",
                          "roa", "netProfitMarginQuarter", "netProfitMarginAccum", "de", "fixedAssetTurnover", "totalAssetTurnover"]
    for key in financial_key_list:
        prop_uri = EX[f"has_{key}"]
        g.add((prop_uri, RDF.type, OWL.DatatypeProperty))
        g.add((prop_uri, RDFS.domain, EX.FinancialStatement))
        g.add((prop_uri, RDFS.range, XSD.float))

    g.add((EX.hasIndustrialClassification, RDF.type, OWL.ObjectProperty))
    g.add((EX.hasIndustrialClassification, RDFS.domain, EX.Company))
    g.add((EX.hasIndustrialClassification, RDFS.range, EX.IndustrialClassification))

    classification_key_list = ['marketCode', 'marketSegment', 'sectorCode']
    for key in classification_key_list:
        prop_uri = EX[f"has_{key}"]
        g.add((prop_uri, RDF.type, OWL.DatatypeProperty))
        g.add((prop_uri, RDFS.domain, EX.IndustrialClassification))
        g.add((prop_uri, RDFS.range, XSD.string))

# Create RDF Graph
def create_rdf_graph(financial_data, eod_data):
    g = Graph()
    g.bind("ex", EX)

    add_schema_definitions(g)
    
    # Helper functions for adding nodes
    def add_stock_value_node(item, g, price_uri_in_dict, price_symbol_in_json):
        stock_value_uri = EX[f"stock_value_{price_symbol_in_json}_{item['date']}"]
        g.add((stock_value_uri, RDF.type, EX.StockValue))
        g.add((price_uri_in_dict, EX.hasStockValue, stock_value_uri))
        
        key_list = ['prior', 'open', 'high', 'low', 'close', 'average', 'totalVolume']
        if item['date'] is not None:
            for key in key_list:
                if key in item and item[key] is not None:
                    g.add((stock_value_uri, EX[key], Literal(item[key], datatype=XSD.float)))
                else:
                    print(f"Missing or None value for {key} in item:", item)
        return

    def add_financial_statement_node(item, g, company_uri_in_dict, price_symbol_in_json):
        year = item['year']
        quarter = item['quarter']
        financial_statement_uri = EX[f"financial_statement_{price_symbol_in_json}_year{year}_quarter{quarter}"]
        g.add((financial_statement_uri, RDF.type, EX.FinancialStatement))
        g.add((company_uri_in_dict, EX.hasFinancialStatement, financial_statement_uri))
        g.add((financial_statement_uri, EX.hasYear, Literal(year, datatype=XSD.integer)))
        g.add((financial_statement_uri, EX.hasQuarter, Literal(quarter, datatype=XSD.integer)))
        
        key_list_float = ["totalAssets", "totalLiabilities", "paidupShareCapital", "shareholderEquity",
                        "totalEquity", "totalRevenueQuarter", "totalRevenueAccum", "totalExpensesQuarter", "totalExpensesAccum",
                        "ebitQuarter", "ebitAccum", "netProfitQuarter", "netProfitAccum", "epsQuarter", "epsAccum",
                        "operatingCashFlow", "investingCashFlow", "financingCashFlow", "roe",
                        "roa", "netProfitMarginQuarter", "netProfitMarginAccum", "de", "fixedAssetTurnover", "totalAssetTurnover"]
        
        for key in key_list_float:
            if key in item and item[key] is not None:
                g.add((financial_statement_uri, EX[key], Literal(item[key], datatype=XSD.float)))
#            else:
#                    print(f"Missing or None value for {key} in item:", item)
        return

    def add_industrial_classification_node(item, g, company_uri_in_dict, price_symbol_in_json):
        industrial_classification_uri = EX[f"industrial_classification_{price_symbol_in_json}"]
        g.add((industrial_classification_uri, RDF.type, EX.IndustrialClassification))
        g.add((company_uri_in_dict, EX.hasIndustrialClassification, industrial_classification_uri))
        
        key_list = ['marketCode', 'marketSegment', 'sectorCode']
        for key in key_list:
            if key in item and item[key] is not None:
                g.add((industrial_classification_uri, EX[key], Literal(item[key])))
        return

    # Create nodes and relationships
    for entry in financial_data:
        company_uri = EX[entry['symbol']]
        add_financial_statement_node(entry, g, company_uri, entry['symbol'])
        add_industrial_classification_node(entry, g, company_uri, entry['symbol'])
    
    for entry in eod_data:
        company_uri = EX[entry['symbol']]
        add_stock_value_node(entry, g, company_uri, entry['symbol'])
    
    return g

# Insert RDF data into Neo4j
def insert_data_into_neo4j(graph, driver):
    with driver.session() as session:
        tx = session.begin_transaction()

        for s, p, o in graph:
            if (p == RDF.type):
                tx.run("MERGE (a:Label {uri: $subject})", subject=str(s))
            elif isinstance(o, Literal):
                predicate = str(p).split('/')[-1]
                query = (
                    "MATCH (a {uri: $subject}) "
                    "SET a." + predicate + " = $object "
                )
                tx.run(query, subject=str(s), object=o.toPython())
            else:
                tx.run(
                    "MERGE (a {uri: $subject}) "
                    "MERGE (b {uri: $object}) "
                    "MERGE (a)-[r:RELATION {type: $predicate}]->(b) ",
                    subject=str(s), predicate=str(p), object=str(o)
                )
                
        tx.commit()

# Main function
def main(financial_file, eod_file):
    financial_data, eod_data = load_json_data(financial_file, eod_file)
    rdf_graph = create_rdf_graph(financial_data, eod_data)
    insert_data_into_neo4j(rdf_graph, driver)

# File paths
financial_file = "C:\\study\\thesis\\Naphattha\\FilteredFinancialData.json"
eod_file = "C:\\study\\thesis\\Naphattha\\FilteredEODData.json"
main(financial_file, eod_file)

Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


Missing or None value for fixedAssetTurnover in item: {'symbol': 'BBL', 'year': '2019', 'quarter': '1', 'financialStatementType': 'C', 'dateAsof': '2019-03-31', 'accountPeriod': 'C', 'totalAssets': 3148836302.0, 'totalLiabilities': 2723140802.0, 'paidupShareCapital': 19088429.0, 'shareholderEquity': 425255994.0, 'totalEquity': 425695500.0, 'totalRevenueQuarter': 38862314.0, 'totalRevenueAccum': 38862314.0, 'totalExpensesQuarter': 30092113.0, 'totalExpensesAccum': 30092113.0, 'ebitQuarter': 21413289.0, 'ebitAccum': 21413289.0, 'netProfitQuarter': 9028296.0, 'netProfitAccum': 9028296.0, 'epsQuarter': 4.73, 'epsAccum': 4.73, 'operatingCashFlow': 9102208.0, 'investingCashFlow': -12584040.0, 'financingCashFlow': 43414.0, 'roe': 8.51157912967361, 'roa': 2.657317017802743, 'netProfitMarginQuarter': 23.475593861960974, 'netProfitMarginAccum': 23.475593861960974, 'de': 6.396921748057003, 'fixedAssetTurnover': None, 'totalAssetTurnover': 0.049008892175058605}
Missing or None value for fixedAsset